### Install Dependencies

In [ ]:
!pip install requests
!pip install xmltodict
!pip install pandas

### Import libraries

In [ ]:
import requests as rq
import xmltodict
import json
import os
from xml.etree import ElementTree
import pandas as pd

### Get All Stops for an Agency

In [ ]:
agency = 'ttc'

AGENCY_DIRECTORY = os.path.join(os.getcwd(),'Agencies',agency)
if not os.path.isdir(AGENCY_DIRECTORY):
    os.makedirs(AGENCY_DIRECTORY)

# Get all routes for agency:

routes_url = 'https://retro.umoiq.com/service/publicXMLFeed?command=routeList&a={AGENCY}'.format(AGENCY=agency)
response = rq.get(routes_url)
resp_dict = xmltodict.parse(response.content)
route_tags = [route['@tag'] for route in resp_dict['body']['route']]

stop_df = pd.DataFrame()


for route in route_tags:
    stops_url = 'https://retro.umoiq.com/service/publicXMLFeed?command=routeConfig&a={AGENCY}&r={ROUTE}'.format(AGENCY=agency, ROUTE=route)
    response = rq.get(stops_url)
    resp_dict = xmltodict.parse(response.content)
    for i,stop in enumerate(resp_dict['body']['route']['stop']):
        if '@stopId' not in stop:
            resp_dict['body']['route']['stop'][i]['@stopId']='NULL'
    stop_ids = [stop['@stopId'] for stop in resp_dict['body']['route']['stop']]
    stop_names = [stop['@title'] for stop in resp_dict['body']['route']['stop']]
    stop_lats = [stop['@lat'] for stop in resp_dict['body']['route']['stop']]
    stop_lons = [stop['@lon'] for stop in resp_dict['body']['route']['stop']]
    route_df = pd.DataFrame()
    route_df['stop_id'], route_df['name'], route_df['lat'], route_df['lon'] = stop_ids, stop_names, stop_lats, stop_lons
    stop_df = pd.concat([stop_df,route_df], ignore_index=True)


stoplist = stop_df['stop_id'].to_list()
unique_inds = [stoplist.index(stop_id) for stop_id in list(set(stoplist))]
stop_df = stop_df.iloc[unique_inds].reset_index(drop=True)

stop_df.to_csv(os.path.join(AGENCY_DIRECTORY,'stop_list.csv')) 



In [ ]:
stop_df.to_csv(os.path.join(AGENCY_DIRECTORY,'stop_list.csv')) 

### Pull Next Arrivals for Stop

In [67]:
import collections

AGENCY_DIRECTORY = os.path.join(os.getcwd(),'Agencies',agency)
stop_df = pd.read_csv(os.path.join(AGENCY_DIRECTORY,'stop_list.csv')) 
for stop_id in stop_df['stop_id'].to_list():
    
    stop_id = str(int(stop_id))

    stop_url = 'https://retro.umoiq.com/service/publicXMLFeed?command=predictions&a={AGENCY}&stopId={STOP}'.format(AGENCY=agency,STOP=stop_id)
    response = rq.get(stop_url)
    resp_dict = xmltodict.parse(response.content)

    next_arrival_resp = dict()
    next_arrival_resp['agency'] = agency
    next_arrival_resp['stop_id'] = stop_id

    next_arrival_resp['routes'] = []

    if type(resp_dict['body']['predictions']) != list:
        routes = [resp_dict['body']['predictions']]
    else:
        routes = [route for route in resp_dict['body']['predictions']]

    i = 0
    for route in routes:
        if 'direction' in route:
            direction = route['direction']
            if type(direction)==list:
                for subdir in direction:
                    next_arrival_resp['routes'].extend([dict()])
                    next_arrival_resp['routes'][i]['route_id'] = subdir['@title'].split(' - ')[1].split(' ')[0]
                    next_arrival_resp['routes'][i]['headsign'] = subdir['@title']
                    if type(subdir['prediction'])==list:
                        next_arrival_resp['routes'][i]['arrival_pred_mins'] = [int(pred['@minutes']) for pred in subdir['prediction']]
                    else:
                        next_arrival_resp['routes'][i]['arrival_pred_mins'] = [int(subdir['prediction']['@minutes'])]
                    i+=1
            else:
                next_arrival_resp['routes'].extend([dict()])
                next_arrival_resp['routes'][i]['route_id'] = route['@routeTag']
                next_arrival_resp['routes'][i]['headsign'] = direction['@title']
                if type(direction['prediction'])==list:
                    next_arrival_resp['routes'][i]['arrival_pred_mins'] = [int(pred['@minutes']) for pred in direction['prediction']]
                else:
                    next_arrival_resp['routes'][i]['arrival_pred_mins'] = [int(direction['prediction']['@minutes'])]
                i+=1
        else:
            next_arrival_resp['routes'].extend([dict()])
            next_arrival_resp['routes'][i]['route_id'] = route['@routeTag']
            next_arrival_resp['routes'][i]['headsign'] = route['@dirTitleBecauseNoPredictions']
            next_arrival_resp['routes'][i]['arrival_pred_mins'] = []
            i+=1
        


{'agency': 'ttc', 'stop_id': '2264', 'routes': [{'route_id': '334', 'headsign': 'East - 334 Eglinton East towards Finch and Neilson via Kingston Rd and Morningside', 'arrival_pred_mins': []}, {'route_id': '34', 'headsign': 'East - 34a Eglinton East towards Kennedy Station', 'arrival_pred_mins': [3, 22, 44, 59]}]}
{'agency': 'ttc', 'stop_id': '15846', 'routes': [{'route_id': '169', 'headsign': 'East - 169a Huntingwood towards Scarborough Centre via Van Horne', 'arrival_pred_mins': [2, 49, 79]}]}
{'agency': 'ttc', 'stop_id': '6771', 'routes': [{'route_id': '66b', 'headsign': 'South - 66b Prince Edward towards Lakeshore', 'arrival_pred_mins': [3, 28, 59]}, {'route_id': '66a', 'headsign': 'South - 66a Prince Edward towards Humber Loop', 'arrival_pred_mins': [24, 43]}]}
{'agency': 'ttc', 'stop_id': '7026', 'routes': [{'route_id': '73', 'headsign': 'North - 73c Royal York towards Claireport via Albion Rd', 'arrival_pred_mins': [0, 19, 39, 59]}, {'route_id': '352', 'headsign': 'West - 352 Law

{'agency': 'ttc', 'stop_id': '1046', 'routes': [{'route_id': '38', 'headsign': 'East - 38a Highland Creek towards Rouge Hill GO Station', 'arrival_pred_mins': [8, 20, 32, 44, 56]}, {'route_id': '95', 'headsign': 'East - 95a York Mills towards Port Union', 'arrival_pred_mins': [19, 49]}, {'route_id': '395', 'headsign': 'East - 395 York Mills towards Meadowvale and Sheppard', 'arrival_pred_mins': []}, {'route_id': '133', 'headsign': 'North - 133 Neilson towards Morningside Heights via Centenary', 'arrival_pred_mins': [7, 18, 29, 40, 51]}, {'route_id': '995', 'headsign': 'East - 995 York Mills Express towards U of T Scarborough', 'arrival_pred_mins': [5, 25, 48]}]}
{'agency': 'ttc', 'stop_id': '11272', 'routes': [{'route_id': '927', 'headsign': 'North - 927b Highway 27 Express towards Steeles via Humber College', 'arrival_pred_mins': [0, 8, 14, 28, 36]}, {'route_id': '96b', 'headsign': 'West - 96b Wilson towards Humberline & Albion', 'arrival_pred_mins': [5, 30, 49]}, {'route_id': '96a', 

{'agency': 'ttc', 'stop_id': '10150', 'routes': [{'route_id': '21', 'headsign': 'South - 21b Brimley towards Scarborough Centre Station', 'arrival_pred_mins': [5, 20, 32, 47]}]}
{'agency': 'ttc', 'stop_id': '4100', 'routes': [{'route_id': '43', 'headsign': 'North - 43a Kennedy towards Steeles', 'arrival_pred_mins': [0, 9, 15, 25, 35]}, {'route_id': '343', 'headsign': 'North - 343 Kennedy towards Steeles', 'arrival_pred_mins': []}]}
{'agency': 'ttc', 'stop_id': '2361', 'routes': [{'route_id': '332', 'headsign': 'West - 332 Eglinton West towards Renforth and Pearson Airport', 'arrival_pred_mins': []}, {'route_id': '32c', 'headsign': 'West - 32c Eglinton West towards Jane & Lawrence via Trethewey', 'arrival_pred_mins': [1, 11, 23, 35, 47]}, {'route_id': '32a', 'headsign': 'West - 32a Eglinton West towards Renforth Station', 'arrival_pred_mins': [8, 20, 32, 44, 56]}]}
{'agency': 'ttc', 'stop_id': '6753', 'routes': [{'route_id': '167', 'headsign': 'South - 167a Pharmacy North towards Don Mi

KeyboardInterrupt: 

In [66]:
direction

[OrderedDict([('@title', 'South - 66b Prince Edward towards Lakeshore'),
              ('prediction',
               [OrderedDict([('@epochTime', '1649693324118'),
                             ('@seconds', '257'),
                             ('@minutes', '4'),
                             ('@isDeparture', 'false'),
                             ('@affectedByLayover', 'true'),
                             ('@branch', '66B'),
                             ('@dirTag', '66_0_66B'),
                             ('@vehicle', '8465'),
                             ('@block', '66_4_40'),
                             ('@tripTag', '43713915')]),
                OrderedDict([('@epochTime', '1649694800885'),
                             ('@seconds', '1734'),
                             ('@minutes', '28'),
                             ('@isDeparture', 'false'),
                             ('@affectedByLayover', 'true'),
                             ('@branch', '66B'),
                             (

In [52]:
next_arrival_resp['routes'][1]['headsign'].split(' - ')[1].split(' ')[0]

'12c'

In [ ]:
def etree_to_dict(t):
    d = {t.tag : map(etree_to_dict, t.iterchildren())}
    d.update(('@' + k, v) for k, v in t.attrib.iteritems())
    d['text'] = t.text
    return d

treedict = etree_to_dict(tree)